http://www.bom.gov.au/climate/data/lists_by_element/alphaAUS_139.txt

In [1]:
import pandas as pd 
rankings_colname = ['data', '1', '2', '3', '4', '5', '6', '7']
data = pd.read_csv('./BOM-Station-Origin.csv', names=rankings_colname)
data = data.dropna(axis=1, how='all')
data = data.dropna(axis=0, how='all')

head = data.loc[:2]
databody = data.loc[4:]

In [2]:
rankings_colname = head.data[2].split()
rankings_colname

['Site', 'Name', 'Lat', 'Lon', 'Start', 'End', 'Years', '%', 'AWS']

In [3]:
def abbbbb(x):
    stationinfo = x.data[:49]
    locinfo = x.data[49:]
    Site = stationinfo.split()[0]
    Name = ' '.join(stationinfo.split()[1:])
    # print(stationId)
    # print(stationName)
    # print(locinfo[:10], locinfo[10:19], locinfo[19:27], locinfo[28:36])
    Lat = locinfo[:10]
    Lon = locinfo[10:19]
    Start = locinfo[19:27]
    End = locinfo[28:36]
    Years = locinfo[36:].split()[0]
    perc = locinfo[36:].split()[1]
    
    try:
        AWS = locinfo[36:].split()[2]
    except:
        AWS = 'None'
        print(x.data)
    d = {'Site': Site, 'Name': Name, 'Lat': Lat, 'Lon': Lon, 'Start':Start, 'End':End, 'Years':Years, '%':perc, 'AWS':AWS}
    return pd.Series(data=d, index=rankings_colname)
    

In [4]:
data = databody.apply(lambda x: abbbbb(x), axis = 1)

  10710 BEAUFORT PLAINS                          -34.3817  118.8233 Jan 1961 Dec 1973   13.0  100    
  94275 CLIFTON BEACH (CAPE DESLACS)             -42.9677  147.5342 Jun 2022 Mar 2024    1.8  100    
  10246 COORINGA                                 -31.6000  116.4697 Apr 1891 Nov 1897    5.6   84    
  12197 DANIEL ROAD                              -32.9833  120.2500 Apr 1963 Apr 1968    1.7   33    
  35027 EMERALD POST OFFICE                      -23.5267  148.1617 Jan 1883 Jun 1992  109.5  100    
   9949 FERRARIS FARM                            -33.4072  116.6242 Jan 1972 May 1986   12.8   89    
  70350 GINNINDERRA (CHARNWOOD (ACT))            -35.2063  149.0113 Jul 2013 Mar 2024   10.3   95    
  94227 GLENORCHY (MURRAYFIELD)                  -42.8442  147.2667 Jul 1898 Aug 1950   52.2  100    
  12192 HYDEN 1                                  -32.2333  120.4000 Jan 1963 Mar 1966    0.6   18    
  12193 HYDEN 2                                  -32.2167  120.5000 Jan 1963 Mar 1

In [5]:
data.to_json('./BOM-Station.json')

In [6]:
data.head()

,Site,Name,Lat,Lon,Start,End,Years,%,AWS
4,41497,AAC DALBY CAMPUS,-27.1494,151.2894,Aug 1985,Sep 2012,26.8,98,N
5,33259,ABBOT POINT BULKCOAL,-19.8833,148.0750,Jan 1991,Jun 1991,0.5,100,N
6,67114,ABBOTSBURY (FAIRFIELD (CITY FARM)),-33.8667,150.8611,Dec 2000,Mar 2024,22.9,98,N
7,40000,ABBOTSFORD,-27.9500,153.1000,Oct 1909,Jul 1974,64.4,99,N
8,86307,ABBOTSFORD,-37.8000,145.0000,May 1972,Sep 1972,0.4,99,N


In [6]:
import geopandas as gpd
import json
from shapely.geometry import Point

#replace it with proper path
file_path = "C:/Users/zheyu/COMP90024_2024_ASMT2_Group12/data/SA2-Map/SA2_2021_AUST_GDA2020.shp"
sa2_gdf = gpd.read_file(file_path)
sa2_gdf = sa2_gdf.to_crs(epsg=4326)

with open('BOM.json', 'r') as json_file:
    data = json.load(json_file)

#GeoDataFrame
records = []
for entry in data:
    lat = float(entry['Location']['lat'])
    lon = float(entry['Location']['lon'])
    point = Point(lon, lat)
    record = {
        'Site': entry['Site'],
        'Name': entry['Name'],
        'Start': entry['Start'],
        'End': entry['End'],
        'Years': entry['Years'],
        '%': entry['%'],
        'AWS': entry['AWS'],
        'Location': entry['Location'],
        'geometry': point
    }
    records.append(record)

gdf = gpd.GeoDataFrame(records, geometry='geometry', crs="EPSG:4326")
gdf_projected = gdf.to_crs(epsg=3857)
sa2_gdf_projected = sa2_gdf.to_crs(epsg=3857)
gdf_with_sa2 = gpd.sjoin_nearest(gdf_projected, sa2_gdf_projected, how='left', distance_col='distance')
#state
gdf_with_sa2['state'] = gdf_with_sa2['STE_NAME21'] 
gdf_with_sa2 = gdf_with_sa2.to_crs(epsg=4326)
#NaN for null
required_columns = ['Site', 'Name', 'Start', 'End', 'Years', '%', 'AWS', 'Location', 'SA2_NAME21', 'state']
gdf_with_sa2 = gdf_with_sa2.reindex(columns=required_columns)
output_data = gdf_with_sa2.to_dict(orient='records')

with open('suburb_centre_bom.json', 'w') as json_output_file:
    json.dump(output_data, json_output_file, indent=2)

print("save successfully")


处理完成并保存
